In [1]:
import numpy as np
import pandas as pd
import csv
import Orange
from orangecontrib.associate.fpgrowth import *

In [2]:
#df = pd.read_csv('./associate.csv').associate.str.replace("[","").str.replace("]","").str.replace("https://filmaga.filmarks.com","").str.replace("'","")
#df.to_csv('./file/in.basket',header=False, index=False, encoding='utf-8')

In [3]:
#df = pd.read_csv('./asso_data_all.csv').associate.str.replace("[","").str.replace("]","").str.replace("'","")
#df = df[~(df.str.contains("/rss") | df.str.contains(".ico"))]
#df.to_csv('./file/in.basket',header=False, index=False, encoding='utf-8')

In [4]:
df = pd.read_csv('./associate_00.csv').associate.str.replace("[","").str.replace("]","").str.replace("'","")
df.to_csv('./file/in.basket',header=False, index=False, encoding='utf-8')

65959

In [5]:
in_file = open('./file/in.basket','r')
out_file = open('./file/out.basket', 'w') 

lines = in_file.readlines()

for line in lines:
    row = line.replace("\"","")
    out_file.write(row)

in_file.close()
out_file.close()

In [6]:
data = Orange.data.Table('./file/out.basket')

In [7]:
X, mapping = OneHot.encode(data)

# frequent~()の第2変数はsupportの最小値.今回は総数の0.1%
itemsets = dict(frequent_itemsets(X, 5))

In [8]:
# 第2変数はconfidenceの最小値.
rules = association_rules(itemsets, 0.7)

In [9]:
stats = rules_stats(rules, itemsets, len(X))

In [10]:
def decode_onehot(d):
    items = OneHot.decode(d, data, mapping)
    return list(map(lambda v: v[1].name, items))

In [11]:
result_file = open('./file/result4.csv', 'w') 

In [12]:
for s in sorted(stats, key = lambda x: x[6], reverse = True):

    lhs = decode_onehot(s[0])
    rhs = decode_onehot(s[1])
    
    if "writers" in str(lhs) or "writers" in str(rhs):

        support = round((s[2]/len(data)), 5)
        confidence = round(s[3], 4)
        lift = round(s[6], 4)
    
        result_file.write(f"lhs = {lhs}, rhs = {rhs}, support = {support}, confidence = {confidence}, lift = {lift}\n")

    else:
        pass
    
result_file.close()